## 1. Building a Simple TCP Server

### Objective

Create a TCP server that accepts client connections, echoes received messages, and can optionally log data.

In [33]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)  # Allow 1 pending connection
print("TCP Server is listening on port 65432...")

try:
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connected to {client_address}")
        data = client_socket.recv(1024)
        print(f"Received: {data.decode()}")
        
        # Echo back the data
        client_socket.sendall(b"ACK: " + data)
        
        # OPTIONAL: Log the received data to a file
        # with open('received_data.txt', 'a', encoding='utf-8') as f:
        #     f.write(f"{client_address}: {data.decode()}\n")
        
        client_socket.close()
except KeyboardInterrupt:
    print("\nServer is shutting down.")
finally:
    server_socket.close()

TCP Server is listening on port 65432...

Server is shutting down.


## 2. TCP Client – Measuring Send Time with datetime

### Objective

Modify the TCP client to measure how long it takes to send data and receive a response.

In [ ]:
import socket
import datetime

# Create a TCP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the server (ensure the server is running on this address and port)
client_socket.connect(('localhost', 65432))

message = input("Enter message: ")

# Measure start time
start_time = datetime.datetime.now()

# Send message to the server
client_socket.sendall(message.encode())

# Receive response from the server
response = client_socket.recv(1024)

# Measure end time
end_time = datetime.datetime.now()

# Calculate round-trip time
elapsed = end_time - start_time

print("Server response:", response.decode())
print("Round-trip time:", elapsed.total_seconds(), "seconds")

client_socket.close()


## 3. UDP Client – Compare Transmission Time

### Objective

Change the script so that the data is sent using UDP instead of TCP. Compare the time taken with TCP.

Note: Run a corresponding UDP server on port 65432 (or change the port as needed).

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65432)

message = input("Enter message (UDP): ")

start_time = datetime.datetime.now()
client_socket.sendto(message.encode(), server_address)

# For UDP, receive response if the server sends one
response, _ = client_socket.recvfrom(1024)
end_time = datetime.datetime.now()
elapsed = end_time - start_time

print(f"Server response: {response.decode()}")
print(f"Time taken (UDP): {elapsed.total_seconds()} seconds")

client_socket.close()

KeyboardInterrupt: 

## 4. Logging Data in a TXT File Over TCP

### Objective

Enhance the TCP server so that every received message is also saved (appended) to a text file.

In [24]:
# The following snippet shows how to log data to a file.
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)
print("TCP Server with logging is listening on port 65432...")

try:
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connected to {client_address}")
        data = client_socket.recv(1024)
        decoded_data = data.decode()
        print(f"Received: {decoded_data}")
        
        # Echo back the data
        client_socket.sendall(b"ACK: " + data)
        
        # Log data to a file
        with open('received_data.txt', 'a', encoding='utf-8') as f:
            f.write(f"{client_address}: {decoded_data}\n")
        
        client_socket.close()
except KeyboardInterrupt:
    print("\nServer is shutting down.")
finally:
    server_socket.close()

TCP Server with logging is listening on port 65432...

Server is shutting down.


## 5. File Transfer Over TCP

### Objective

Transfer a file from client to server over TCP. Create a text file named `file_to_send.txt` in the same folder as your notebook.

### TCP File Transfer – Client

In [25]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

start_time = datetime.datetime.now()
with open('file_to_send.txt', 'rb') as f:
    client_socket.sendfile(f)
end_time = datetime.datetime.now()
elapsed = end_time - start_time

print("File sent over TCP!")
print(f"Time taken: {elapsed.total_seconds()} seconds")
client_socket.close()

ConnectionRefusedError: [Errno 61] Connection refused

### TCP File Transfer – Server

The server code below receives file data and writes it to `received_file.txt`.

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)
print("TCP File Server is listening on port 65432...")

client_socket, client_address = server_socket.accept()
print(f"Connected to {client_address}")

with open('received_file.txt', 'wb') as f:
    while True:
        data = client_socket.recv(1024)
        if not data:
            break
        f.write(data)

print("File received!")
client_socket.close()
server_socket.close()

TCP File Server is listening on port 65432...


KeyboardInterrupt: 

## 6. File Transfer Over UDP (Variation)

### UDP File Transfer – Client

For UDP file transfer, send the entire file data in one go (note that UDP does not guarantee delivery or order).

In [ ]:
import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

with open('file_to_send.txt', 'rb') as f:
    file_data = f.read()

start_time = datetime.datetime.now()
client_socket.sendto(file_data, server_address)
end_time = datetime.datetime.now()
elapsed = end_time - start_time

print("File sent over UDP!")
print(f"Time taken: {elapsed.total_seconds()} seconds")
client_socket.close()

FileNotFoundError: [Errno 2] No such file or directory: 'file_to_send.txt'

### UDP File Transfer – Server

The UDP server below listens on port 65433 and writes the received data to `received_file_udp.txt`.

In [28]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))
print("UDP File Server is listening on port 65433...")

data, client_address = server_socket.recvfrom(65535)  # large buffer size
with open('received_file_udp.txt', 'wb') as f:
    f.write(data)

print("File received over UDP!")
server_socket.close()

OSError: [Errno 48] Address already in use

## 7. Challenge: TCP-Based Chat System

### Objective

Design a TCP-based chat system where a central server relays messages between multiple clients in real time using threading.

### Chat Server Code (Using Threading)

In [30]:
import socket
import threading

# List to keep track of client sockets
clients = []

def handle_client(client_socket, client_address):
    print(f"New connection: {client_address}")
    try:
        while True:
            message = client_socket.recv(1024)
            if not message:
                break
            broadcast(message, client_socket)
    except Exception as e:
        print(f"Error with {client_address}: {e}")
    finally:
        print(f"Connection closed: {client_address}")
        clients.remove(client_socket)
        client_socket.close()

def broadcast(message, sender_socket):
    for client in clients:
        if client != sender_socket:
            try:
                client.sendall(message)
            except:
                pass

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(5)
print("Chat server listening on port 65432...")

try:
    while True:
        client_socket, client_address = server_socket.accept()
        clients.append(client_socket)
        thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
        thread.daemon = True
        thread.start()
except KeyboardInterrupt:
    print("Shutting down chat server...")
finally:
    server_socket.close()

OSError: [Errno 48] Address already in use

### Chat Client Code

In [31]:

import socket
import threading

def receive_messages(client_socket):
    while True:
        try:
            message = client_socket.recv(1024)
            if not message:
                break
            print(message.decode())
        except:
            break

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

# Start thread for receiving messages
thread = threading.Thread(target=receive_messages, args=(client_socket,))
thread.daemon = True
thread.start()

print("Connected to chat server. Type messages and press Enter to send. Type 'exit' to quit.")
try:
    while True:
        msg = input()
        if msg.lower() == 'exit':
            break
        client_socket.sendall(msg.encode())
except KeyboardInterrupt:
    pass
finally:
    client_socket.close()

Connected to chat server. Type messages and press Enter to send. Type 'exit' to quit.


## 8. Challenge: Adding Encryption for Secure Messaging

### Objective

Integrate encryption into your chat system so that messages are encrypted before sending and decrypted on receipt. The example below uses the Fernet module from the `cryptography` library.

Before running, install the library using:
```
pip install cryptography
```

In [32]:

from cryptography.fernet import Fernet

# Generate a key (do this once and save the key securely)
key = Fernet.generate_key()
print("Key:", key)

# Create a Fernet cipher instance
cipher = Fernet(key)

# Encrypt a message
original_message = "Hello, secure world!"
encrypted_message = cipher.encrypt(original_message.encode())
print("Encrypted:", encrypted_message)

# Decrypt the message
decrypted_message = cipher.decrypt(encrypted_message).decode()
print("Decrypted:", decrypted_message)

ModuleNotFoundError: No module named 'cryptography'